In [1]:
import sys
sys.path.insert(0,'/nfs/users/xueyou/github/models')

import os
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [2]:
import io
import numpy as np
import tensorflow as tf

In [3]:
cd /nfs/users/xueyou/github/Bert-for-Chinese-NLP/

/nfs/users/xueyou/github/Bert-for-Chinese-NLP


In [4]:
from model import siamese_bert
from official.nlp.bert import configs as bert_configs

In [5]:
BERT_DIR='/data/xueyou/data/bert_pretrain/converted/chinese_L-12_H-768_A-12'
bert_config = bert_configs.BertConfig.from_json_file(BERT_DIR+'/bert_config.json')

In [6]:
bert_siamese, encoder = siamese_bert.siamese_model(bert_config,2,'triplet')

In [7]:
model = bert_siamese.encoder

In [8]:
model.inputs

[<tf.Tensor 'input_word_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'input_mask:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'input_type_ids:0' shape=(None, None) dtype=int32>]

In [9]:
model.outputs

[<tf.Tensor 'dropout_1/cond/Identity:0' shape=(None, 768) dtype=float32>]

In [10]:
bert_siamese.load_weights('/data/xueyou/data/corpus/task_data/lcqmc/triplet/best/model')

In [11]:
model = bert_siamese.encoder

In [12]:
from official.nlp.bert import model_saving_utils, tokenization

tokenizer = tokenization.FullTokenizer(BERT_DIR + '/vocab.txt', do_lower_case=True)

In [13]:
from data import dataset,input_pipeline

In [14]:
class MyData(dataset.TripletDataset):
    def filter_example(self, item):
        return False

In [15]:
test_data = MyData(
    tokenizer, 
    '/data/xueyou/data/corpus/task_data/lcqmc/test.json', 
    'eval', 
    './labels/binary.txt', 
    128)
dev_data = MyData(
    tokenizer, 
    '/data/xueyou/data/corpus/task_data/lcqmc/dev.json', 
    'eval', 
    './labels/binary.txt', 
    128)

In [16]:
test_dataset = input_pipeline.create_siamese_dataset(test_data,64,False,None)
dev_dataset = input_pipeline.create_siamese_dataset(dev_data,64,False,None)

In [17]:
max_seq_length=128
BERT_UNKOWN_TOKEN = "[UNK]"
BERT_CLS_TOKEN = "[CLS]"
BERT_SEP_TOKEN = "[SEP]"
BERT_MASK_TOKEN = "[MASK]"
def encode_single_sentence(sentence, segment=0):
    tokens = list(tokenizer.tokenize(sentence))
    # Account for CLS and SEP
    tokens = tokens[0:max_seq_length-2]
    tokens = [BERT_CLS_TOKEN] + tokens + [BERT_SEP_TOKEN]
    mask = [1] * len(tokens)
    return {
      'input_word_ids': np.asarray([tokenizer.convert_tokens_to_ids(tokens)]),
      'input_mask': np.asarray([mask]),
      'input_type_ids': np.asarray([[segment] * len(tokens)])}

In [18]:
import json

In [19]:
from annoy import AnnoyIndex

In [20]:
f = 768
ann = AnnoyIndex(f, 'euclidean') 
cnt = 0

In [21]:
from tqdm import tqdm
from collections import defaultdict

In [22]:
texts = defaultdict(set)
raw_texts = []

In [23]:
negtives = {}

for item in tqdm(open('/data/xueyou/data/corpus/task_data/lcqmc/test.json')):
    item = json.loads(item)
    raw_texts.append(item['text_a'])
    raw_texts.append(item['text_b'])
    if item['label'] == '0':
        negtives[item['text_a']] = item['text_b']
    texts[item['text_a']].add(item['text_b'])
    texts[item['text_b']].add(item['text_a'])
    
for item in tqdm(open('/data/xueyou/data/corpus/task_data/lcqmc/dev.json')):
    item = json.loads(item)
    raw_texts.append(item['text_a'])
    raw_texts.append(item['text_b'])
    if item['label'] == '0':
        negtives[item['text_a']] = item['text_b']
    texts[item['text_a']].add(item['text_b'])
    texts[item['text_b']].add(item['text_a'])

12500it [00:00, 67309.26it/s]
8802it [00:00, 56396.91it/s]


In [24]:
keys = list(texts.keys())

In [25]:
cc = 0
for x in tqdm(test_dataset):
    (x1,x2),y = x
    cc += len(y)
    vals = model(x1)
    vals2 = model(x2)
    for v1,v2 in zip(vals.numpy(),vals2.numpy()):
        ann.add_item(cnt,v1)
        cnt += 1
        ann.add_item(cnt,v2)
        cnt += 1

for x in tqdm(dev_dataset):
    (x1,x2),y = x
    cc += len(y)
    vals = model(x1)
    vals2 = model(x2)
    for v1,v2 in zip(vals.numpy(),vals2.numpy()):
        ann.add_item(cnt,v1)
        cnt += 1
        ann.add_item(cnt,v2)
        cnt += 1

196it [00:39,  4.91it/s]
138it [00:30,  4.56it/s]


In [26]:
ann.build(10) # 10 trees

True

In [30]:
idx = 0

keys[idx]

'谁有狂三这张高清的'

In [75]:
def get_triplet_sims(idx,text=''):
    if text:
        v = model(encode_single_sentence(text))[0]
    else:
        v = model(encode_single_sentence(keys[idx]))[0]
    cands = set()
    for x,y in zip(*ann.get_nns_by_vector(v.numpy(), 10, -1,include_distances=True)):
        w = raw_texts[x]
        if w in cands:
            continue
        cands.add(w)
        print(w,y)
    

In [31]:
get_triplet_sims(idx)

谁有狂三这张高清的 1.077336219168501e-05
这张高清图，谁有 6.200240612030029
求这张图的高清大图~ 6.374392509460449
求这张动漫高清大图… 6.45466947555542
求艋胛这张图片的高清大图 6.558984279632568
求这张图的高清大图 6.5802693367004395
这张图的高清大图谁有 6.615054130554199
有这张图的高清大图吗 6.805208683013916
求这张动漫图高清大图！ 6.8919854164123535
求这个图片的电脑壁纸～要高清 6.9003424644470215


In [32]:
texts[keys[idx]]

{'这张高清图，谁有'}

In [33]:
negtives.get(keys[idx],'无')

'这张高清图，谁有'

In [34]:
ams_siamese, _ = siamese_bert.siamese_model(bert_config,2,'ams')

In [35]:
ams_siamese.load_weights('/data/xueyou/data/corpus/task_data/lcqmc/ams/best/model')

In [36]:
ams_model = ams_siamese.encoder

In [37]:
f = 768
ann2 = AnnoyIndex(f, 'dot') 
cnt = 0

In [38]:
cc = 0
for x in tqdm(test_dataset):
    (x1,x2),y = x
    cc += len(y)
    vals = ams_model(x1)
    vals2 = ams_model(x2)
    for v1,v2 in zip(vals.numpy(),vals2.numpy()):
        ann2.add_item(cnt,v1)
        cnt += 1
        ann2.add_item(cnt,v2)
        cnt += 1

for x in tqdm(dev_dataset):
    (x1,x2),y = x
    cc += len(y)
    vals = ams_model(x1)
    vals2 = ams_model(x2)
    for v1,v2 in zip(vals.numpy(),vals2.numpy()):
        ann2.add_item(cnt,v1)
        cnt += 1
        ann2.add_item(cnt,v2)
        cnt += 1

196it [00:32,  6.01it/s]
138it [00:25,  5.45it/s]


In [39]:
ann2.build(10)

True

In [74]:
def get_ams_sims(idx,text=''):
    if text:
        v = ams_model(encode_single_sentence(text))[0]
    else:
        v = ams_model(encode_single_sentence(keys[idx]))[0]
    cands = set()
    for x,y in zip(*ann2.get_nns_by_vector(v.numpy(), 10, -1, include_distances=True)):
        w = raw_texts[x]
        if w in cands:
            continue
        cands.add(w)
        print(w,y)
    

In [46]:
import random

In [76]:
for i in random.sample(list(range(len(keys))),5):
    print('+'*20)
    print('原始query -',keys[i])
    print('真实相似  -',texts[keys[i]]) 
    print('反例      -',negtives.get(keys[i],'无'))
    print('#'*20)
    get_triplet_sims(i)
    print('='*20)
    get_ams_sims(i)


++++++++++++++++++++
原始query - 男的，短发能做什么发型呢？
真实相似  - {'短发到肩，什么发型好？'}
反例      - 短发到肩，什么发型好？
####################
男的，短发能做什么发型呢？ 9.351257176604122e-06
短发到肩，什么发型好？ 4.331512928009033
男生头发软适合烫什么发型？ 4.906976222991943
短发烫什么发型好看 4.93504524230957
男头发少适合什么发型 5.176742076873779
男生长发弄什么发型好看？ 5.197969913482666
中短发烫什么发型好看？ 5.229706764221191
男生头发软适合留什么发型 5.305871963500977
想把头发剪短，适合什么发型呢？ 5.351088047027588
我这样的头发剪什么发型好看！ 5.354901313781738
男的，短发能做什么发型呢？ 1.0
男生长发弄什么发型好看？ 0.9980127811431885
男生什么长发发型好看？ 0.9972010254859924
适合做什么发型 0.9970504641532898
求男孩不烫发好看一些的发型 0.9968783855438232
谁知道这个发型怎么扎 0.9967594742774963
做什么发型 0.9965689778327942
谁知道这个发型是怎么梳的啊 0.9962197542190552
++++++++++++++++++++
原始query - 有什么适合女生玩的游戏
真实相似  - {'上海有什么地方适合情侣玩的'}
反例      - 上海有什么地方适合情侣玩的
####################
有什么适合女生玩的游戏 1.1066676961490884e-05
现在什么游戏好玩啊？来个朋友推荐下。 7.59814453125
什么游戏适合上班族玩？ 7.6697235107421875
谁能推荐我几个好玩的小游戏？ 7.91718864440918
推荐几个好玩的小游戏。 8.277565002441406
推荐几个好玩的小游戏，越多越好 8.634293556213379
什么舞蹈适合女生跳？ 9.17080020904541
养宠物的游戏 9.

In [63]:
get_ams_sims(0,'你今天吃药了吗')

晚上尿多吃什么药 0.9857357740402222
糖尿病人能吃什么 0.9809780716896057
糖尿病人吃什么好 0.9809621572494507
刚感冒吃什么药啊 0.9778760075569153
肝病吃什么会好 0.9777190089225769
感冒了吃啥最好？ 0.9771125912666321
糖尿病人感冒吃什么好 0.9769859910011292


In [64]:
get_triplet_sims(0,'你今天吃药了吗')

今天早上没吃药，感觉自己萌萌哒 7.085054397583008
晚上尿多吃什么药 7.133637428283691
今天没吃药感觉自己萌萌哒！ 7.230893611907959
感冒了吃啥最好？ 7.69866418838501
被猫咬了要打针吗 7.723515510559082
今天中午吃什么 7.772645473480225
什么药可以让人昏迷 7.775844097137451
隔夜蛋可以吃吗 7.888108253479004


In [77]:
np.dot(ams_model(encode_single_sentence('你今天吃药了吗'))[0],ams_model(encode_single_sentence('今天药有没有服用'))[0])

0.87022275

In [78]:
np.linalg.norm(model(encode_single_sentence('你今天吃药了吗'))[0]-model(encode_single_sentence('今天药有没有服用'))[0])

4.5063596